## Preparar la base de datos

In [1]:
# código funciona perfectamente
import os
import csv

def cargar_lineas_validas(csv_path, salto=34):
    lineas = []
    with open(csv_path, 'r', encoding='utf-8', errors='ignore') as f:
        for i, line in enumerate(f):
            if i >= salto:
                if line.strip() == "":
                    break
                lineas.append(line.strip())
    return lineas

def encontrar_ruta_componente(archivo, dirs_prioridad):
    for d in dirs_prioridad:
        posible = os.path.join(d, archivo)
        if os.path.exists(posible):
            return posible
    return None

def procesar_vs30(tipo, magnitud, vs30, vs30_path, registros_validos):
    componentes = {
        "H1": os.path.join(vs30_path, "Componente Horizontal 1"),
        "H2": os.path.join(vs30_path, "Componente Horizontal 2"),
        "V":  os.path.join(vs30_path, "Componente Vertical")
    }
    search_csv = os.path.join(vs30_path, "_SearchResults.csv")
    if not os.path.isfile(search_csv):
        print(f"⚠️ No se encuentra el archivo CSV: {search_csv}")
        return 0

    lineas = cargar_lineas_validas(search_csv)
    if not lineas:
        print(f"⚠️ No se encontraron líneas válidas en el CSV: {search_csv}")
        return 0

    validos = 0
    for linea in lineas:
        row = linea.split(",")
        if len(row) < 3:
            continue
        arch_h1, arch_h2, arch_v = row[-3].strip(), row[-2].strip(), row[-1].strip()
        ruta_h1 = encontrar_ruta_componente(arch_h1, [componentes["H1"], componentes["H2"], componentes["V"]])
        ruta_h2 = encontrar_ruta_componente(arch_h2, [componentes["H2"], componentes["H1"]])
        ruta_v  = encontrar_ruta_componente(arch_v,  [componentes["V"],  componentes["H1"], componentes["H2"]])
        
        if ruta_h1 and ruta_h2 and ruta_v:
            validos += 1
            registros_validos.append({
                "Tipo de evento": tipo,
                "Magnitud": magnitud,
                "Vs30": vs30,
                "Archivo H1": arch_h1,
                "Ruta H1": ruta_h1,
                "Archivo H2": arch_h2,
                "Ruta H2": ruta_h2,
                "Archivo V": arch_v,
                "Ruta V": ruta_v
            })
        else:
            print("❌ Faltan archivos:", end=" ")
            if not ruta_h1: print("H1", end=" ")
            if not ruta_h2: print("H2", end=" ")
            if not ruta_v:  print("V",  end=" ")
            print(f"→ en {vs30_path}")
    return validos

def contar_eventos_validos_y_guardar_csv(base_path, tipos_evento, archivo_salida="eventos_validos.csv"):
    total_global = 0
    registros_validos = []

    for tipo in tipos_evento:
        tipo_path = os.path.join(base_path, tipo)
        if not os.path.isdir(tipo_path):
            print(f"❌ No existe la carpeta de tipo: {tipo_path}")
            continue

        for magnitud in os.listdir(tipo_path):
            mag_path = os.path.join(tipo_path, magnitud)
            if not os.path.isdir(mag_path):
                continue

            for vs30 in os.listdir(mag_path):
                vs30_path = os.path.join(mag_path, vs30)
                if not os.path.isdir(vs30_path):
                    continue

                total_local = procesar_vs30(tipo, magnitud, vs30, vs30_path, registros_validos)
                total_global += total_local

                estado = "✅" if total_local > 0 else "⚠️"
                print(f"📁 {tipo} | {magnitud} | {vs30} → {estado} {total_local} eventos válidos")

    # Guardar CSV
    if registros_validos:
        campos = list(registros_validos[0].keys())
        with open(archivo_salida, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=campos)
            writer.writeheader()
            writer.writerows(registros_validos)
        print(f"\n📄 Archivo CSV generado: {archivo_salida}")
    else:
        print("\n⚠️ No se generó ningún registro válido, no se creó el CSV.")

    return total_global

# Ejecutar
base_dir = os.path.join(os.getcwd(), "Base-sismos-2024")
tipos = ["1 Stiker Slip (SS)", "2 Normal-Oblique", "3 Reverse-Oblique"]
total = contar_eventos_validos_y_guardar_csv(base_dir, tipos)
print("\n🎯 TOTAL GLOBAL de eventos válidos con los tres componentes:", total)


❌ Faltan archivos: V → en /home/mare/Documents/Documentos/6. Sexto Semestre/Machine Learning/Proyecto3/Base-sismos-2024/1 Stiker Slip (SS)/Mag. 6-8/Vs30. 0-200
❌ Faltan archivos: H1 → en /home/mare/Documents/Documentos/6. Sexto Semestre/Machine Learning/Proyecto3/Base-sismos-2024/1 Stiker Slip (SS)/Mag. 6-8/Vs30. 0-200
❌ Faltan archivos: H1 → en /home/mare/Documents/Documentos/6. Sexto Semestre/Machine Learning/Proyecto3/Base-sismos-2024/1 Stiker Slip (SS)/Mag. 6-8/Vs30. 0-200
❌ Faltan archivos: H1 → en /home/mare/Documents/Documentos/6. Sexto Semestre/Machine Learning/Proyecto3/Base-sismos-2024/1 Stiker Slip (SS)/Mag. 6-8/Vs30. 0-200
❌ Faltan archivos: H1 → en /home/mare/Documents/Documentos/6. Sexto Semestre/Machine Learning/Proyecto3/Base-sismos-2024/1 Stiker Slip (SS)/Mag. 6-8/Vs30. 0-200
❌ Faltan archivos: H1 → en /home/mare/Documents/Documentos/6. Sexto Semestre/Machine Learning/Proyecto3/Base-sismos-2024/1 Stiker Slip (SS)/Mag. 6-8/Vs30. 0-200
❌ Faltan archivos: H1 → en /home/ma

In [2]:
#Imports
import pandas as pd
import numpy as np
import re
import numpy as np
from scipy.interpolate import interp1d
import ast

In [3]:
df_completo = pd.read_csv('eventos_validos.csv', encoding='utf-8')

In [4]:
# Creación del ID

ph1=df_completo['Archivo H1'].str.split('_').str[:2].str.join('_')
ph2=df_completo['Archivo H2'].str.split('_').str[:2].str.join('_')
phv=df_completo['Archivo V'].str.split('_').str[:2].str.join('_')

mascara = (ph1 == ph2) & (ph1 == phv)
df_completo.loc[mascara, 'Evento ID'] = ph1[mascara]
columnas = ['Evento ID']+ [c for c in df_completo.columns if c != 'Evento ID']
df_completo = df_completo[columnas] #Reordenar, luego esto se lo meteria en el codigo de Andres para que eeste mas clean

#Nose como funciono esto, pero chat ayudo
ocurrencias = df_completo.groupby('Evento ID').cumcount()
mask_segundas = ocurrencias == 1

# 3) Renombra sólo esas filas, añadiendo "_2" al final
df_completo.loc[mask_segundas, 'Evento ID'] = (
    df_completo.loc[mask_segundas, 'Evento ID'] + '_2'
)

df_completo['Evento ID'].nunique()
df_completo.to_csv('eventos_actualizado.csv', index=False) #OJO A LOS NOMBRES DE ARCHIVO


In [5]:
df_completo

,Evento ID,Tipo de evento,Magnitud,Vs30,Archivo H1,Ruta H1,Archivo H2,Ruta H2,Archivo V,Ruta V
0,RSN170_IMPVALL.H,1 Stiker Slip (SS),Mag. 6-8,Vs30. 0-200,RSN170_IMPVALL.H_H-ECC002.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN170_IMPVALL.H_H-ECC092.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN170_IMPVALL.H_H-ECC-UP.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...
1,RSN174_IMPVALL.H,1 Stiker Slip (SS),Mag. 6-8,Vs30. 0-200,RSN174_IMPVALL.H_H-E11140.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN174_IMPVALL.H_H-E11230.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN174_IMPVALL.H_H-E11-UP.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...
2,RSN175_IMPVALL.H,1 Stiker Slip (SS),Mag. 6-8,Vs30. 0-200,RSN175_IMPVALL.H_H-E12140.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN175_IMPVALL.H_H-E12230.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN175_IMPVALL.H_H-E12-UP.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...
3,RSN178_IMPVALL.H,1 Stiker Slip (SS),Mag. 6-8,Vs30. 0-200,RSN178_IMPVALL.H_H-E03140.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN178_IMPVALL.H_H-E03230.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN178_IMPVALL.H_H-E03-UP.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...
4,RSN192_IMPVALL.H,1 Stiker Slip (SS),Mag. 6-8,Vs30. 0-200,RSN192_IMPVALL.H_H-WSM090.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN192_IMPVALL.H_H-WSM180.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN192_IMPVALL.H_H-WSM-UP.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...
...,...,...,...,...,...,...,...,...,...,...
1691,RSN8853_14383980,3 Reverse-Oblique,Mag. 4-6,Vs30. 600-,RSN8853_14383980_CISDRHNN.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN8853_14383980_CISDRHNE.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN8853_14383980_CISDRHNZ.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...
1692,RSN8858_14383980,3 Reverse-Oblique,Mag. 4-6,Vs30. 600-,RSN8858_14383980_CISMRHNN.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN8858_14383980_CISMRHNE.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN8858_14383980_CISMRHNZ.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...
1693,RSN8860_14383980,3 Reverse-Oblique,Mag. 4-6,Vs30. 600-,RSN8860_14383980_CISPFHNN.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN8860_14383980_CISPFHNE.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN8860_14383980_CISPFHNZ.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...
1694,RSN8864_14383980,3 Reverse-Oblique,Mag. 4-6,Vs30. 600-,RSN8864_14383980_CISVDHNN.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN8864_14383980_CISVDHNE.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...,RSN8864_14383980_CISVDHNZ.AT2,/home/mare/Documents/Documentos/6. Sexto Semes...


In [6]:
#Procesamiento de señales
def processNGAfile3(filepath, scalefactor=None):
    """
    Lee un .AT2 y devuelve (npts, dt, time, inp_acc), buscando dinámicamente
    la línea con NPTS= y DT=, y detectando el inicio de los datos numéricos.
    """
    if scalefactor is None:
        scalefactor = 1.0

    with open(filepath, 'r') as f:
        lines = f.readlines()

    # 1) Buscar NPTS y DT en las primeras 20 líneas
    npts = dt = None
    for line in lines[:20]:
        m_n = re.search(r'NPTS\s*=\s*([0-9]+)', line)
        m_d = re.search(r'DT\s*=\s*([0-9]*\.?[0-9]+)', line)
        if m_n: npts = int(m_n.group(1))
        if m_d: dt   = float(m_d.group(1))
        if npts is not None and dt is not None:
            break
    if npts is None or dt is None:
        raise ValueError(f"No se encontró NPTS/DT en {filepath}")

    # 2) Detectar el índice donde empiezan los datos (línea con floats)
    start_idx = None
    num_pattern = re.compile(r'^[-+]?[0-9]*\.?[0-9]+')
    for i, line in enumerate(lines):
        tokens = line.strip().split()
        # si TODOS los tokens tienen formato numérico, asumimos datos
        if tokens and all(num_pattern.match(t) for t in tokens):
            start_idx = i
            break
    if start_idx is None:
        raise ValueError(f"No se encontró inicio de datos en {filepath}")

    # 3) Leer todos los valores desde start_idx en adelante
    acc_data = []
    for line in lines[start_idx:]:
        for tok in line.strip().split():
            acc_data.append(float(tok) * scalefactor)

    inp_acc = np.array(acc_data, dtype=float)
    time    = np.arange(len(acc_data)) * dt

    return npts, dt, time, inp_acc

def agregar_dt_npts(df, comp_cols=('Ruta H1','Ruta H2','Ruta V')):
    """
    A un DataFrame que ya contiene rutas completas en comp_cols,
    le añade dos columnas por componente: NPTS_<sufijo> y DT_<sufijo>.

    Parámetros
    ----------
    df : pd.DataFrame
        Debe tener al menos las columnas indicadas en comp_cols, con la ruta
        completa al archivo .AT2 en cada celda.
    comp_cols : tuple de str
        Nombres de las columnas de df con las rutas para H1, H2 y V.

    Devuelve
    -------
    pd.DataFrame
        Copia de df con las 6 columnas adicionales.
    """
    df2 = df.copy()

    for comp in comp_cols:
        npts_list = []
        dt_list   = []

        # Procesa cada ruta de archivo directamente
        for filepath in df2[comp]:
            # Llama a tu parser robusto:
            npts, dt,_,_ = processNGAfile3(filepath)
            npts_list.append(int(npts))
            dt_list.append(float(dt))

        # Extrae sufijo 'H1','H2' o 'V' a partir del nombre de la columna
        suf = comp.split()[-1]

        # Añade las nuevas columnas
        df2[f'NPTS_{suf}'] = npts_list
        df2[f'DT_{suf}']   = dt_list

    return df2



In [7]:
df_completo=agregar_dt_npts(df_completo,comp_cols=('Ruta H1','Ruta H2','Ruta V'))

In [8]:

# Lista con todos los nuevos nombres, en el orden actual de las columnas
df_completo.columns = ['Evento ID', 'Tipo de evento', 'Magnitud', 'Vs30',
              'Archivo H2', 'Ruta H2', 'Archivo H1', 'Ruta H1',
              'Archivo V', 'Ruta V', 'NPTS_H2', 'DT_H2', 'NPTS_H1', 'DT_H1', 'NPTS_HV', 'DT_HV']

In [9]:
#Normalizacion
def normalizar_frecuencia_muestreo(tiempo, aceleracion, dt_obj=0.01):
    t0, t1 = tiempo[0], tiempo[-1]
    t_new   = np.arange(t0, t1, dt_obj)
    f       = interp1d(tiempo, aceleracion,
                       kind='linear',
                       fill_value='extrapolate')
    acc_new = f(t_new)
    return t_new, acc_new


def estandarización(df, comp_cols=('Ruta H1','Ruta H2','Ruta V'),
                          parser=processNGAfile3,
                          dt_obj=0.01):

    for label in ['H1','H2','V']:
        df[f'Acc_{label}_std'] = None
    df['NPTS_common'] = None
    df['DT_common']   = None

    # 2) Recorre cada evento (fila) de df
    for idx, row in df.iterrows():
        remuestreos = {}

        # 2a) Leer y remuestrear
        for label, col in zip(['H1','H2','V'], comp_cols):
            npts, dt, tiempo, acc = parser(row[col])
            # avisar si el dt original no es el esperado
            if abs(dt - dt_obj) > 1e-8:
                print(f"[Aviso] dt original {dt}≠{dt_obj} en fila {idx} / {label}")
            # remuestreo
            _, acc_norm = normalizar_frecuencia_muestreo(tiempo, acc, dt_obj=dt_obj)
            remuestreos[label] = acc_norm

        # 2b) Determinar longitud mínima común
        min_len = min(len(a) for a in remuestreos.values())
        df.at[idx, 'NPTS_common'] = min_len
        df.at[idx, 'DT_common']   = dt_obj

        # 2c) Truncar y guardar en df
        for label in ['H1','H2','V']:
            df.at[idx, f'Acc_{label}_std'] = remuestreos[label][:min_len].tolist()

    return df

In [10]:
df_completo = estandarización(df_completo,
                                           comp_cols=('Ruta H1','Ruta H2','Ruta V'),
                                           parser=processNGAfile3,dt_obj=0.01)



[Aviso] dt original 0.005≠0.01 en fila 0 / H1
[Aviso] dt original 0.005≠0.01 en fila 0 / H2
[Aviso] dt original 0.005≠0.01 en fila 0 / V
[Aviso] dt original 0.005≠0.01 en fila 1 / H1
[Aviso] dt original 0.005≠0.01 en fila 1 / H2
[Aviso] dt original 0.005≠0.01 en fila 1 / V
[Aviso] dt original 0.005≠0.01 en fila 2 / H1
[Aviso] dt original 0.005≠0.01 en fila 2 / H2
[Aviso] dt original 0.005≠0.01 en fila 2 / V
[Aviso] dt original 0.005≠0.01 en fila 3 / H1
[Aviso] dt original 0.005≠0.01 en fila 3 / H2
[Aviso] dt original 0.005≠0.01 en fila 3 / V
[Aviso] dt original 0.005≠0.01 en fila 4 / H1
[Aviso] dt original 0.005≠0.01 en fila 4 / H2
[Aviso] dt original 0.005≠0.01 en fila 4 / V
[Aviso] dt original 0.005≠0.01 en fila 5 / H1
[Aviso] dt original 0.005≠0.01 en fila 5 / H2
[Aviso] dt original 0.005≠0.01 en fila 5 / V
[Aviso] dt original 0.005≠0.01 en fila 6 / H1
[Aviso] dt original 0.005≠0.01 en fila 6 / H2
[Aviso] dt original 0.005≠0.01 en fila 6 / V
[Aviso] dt original 0.005≠0.01 en fila 7 

In [11]:
#FTT
def FFT_featuring(aceleracion): 

    FFT = np.fft.rfft(aceleracion, n=512) 
    magnitud = np.abs(FFT)

    return magnitud

In [12]:
df_completo['mag_H1'] = df_completo['Acc_H1_std'].apply(lambda x: FFT_featuring(x))
df_completo['mag_H2'] = df_completo['Acc_H2_std'].apply(lambda x: FFT_featuring(x))
df_completo['mag_V']  = df_completo['Acc_V_std'].apply(lambda x: FFT_featuring(x))

# 3. Concatenar las tres magnitudes en una columna de 'features'
df_completo['features_sep'] = df_completo.apply(
    lambda r: np.concatenate([r['mag_H1'], r['mag_H2'], r['mag_V']]),
    axis=1
)

In [13]:
df_pre = df_completo.copy()
df_pre.drop(columns=[ 'Archivo H2',
       'Ruta H2', 'Archivo H1', 'Ruta H1', 'Archivo V', 'Ruta V', 'NPTS_H2',
       'DT_H2', 'NPTS_H1', 'DT_H1', 'NPTS_HV', 'DT_HV', 'Acc_H1_std',
       'Acc_H2_std', 'Acc_V_std', 'NPTS_common', 'DT_common'], inplace=True)

In [14]:
# Cargar el archivo de eventos válidos
eventos_df = pd.read_csv("eventos_validos.csv")

# Definir las posibles clases
tipos_evento = ["1 Stiker Slip (SS)", "2 Normal-Oblique", "3 Reverse-Oblique"]
magnitudes = ["Mag. 4-6", "Mag. 6-8"]
vs30_clases = ["Vs30. 0-200", "Vs30. 200-400", "Vs30. 400-600", "Vs30. 600-"]

# Crear un DataFrame vacío para los labels
labels = pd.DataFrame()

# Crear columnas para cada etiqueta
for tipo in tipos_evento:
    labels[tipo] = (eventos_df["Tipo de evento"] == tipo).astype(int)

for mag in magnitudes:
    labels[mag] = (eventos_df["Magnitud"] == mag).astype(int)

for vs in vs30_clases:
    labels[vs] = (eventos_df["Vs30"] == vs).astype(int)

# Guardar el archivo de labels
labels.to_csv("labels_eventos.csv", index=False)

print("✅ Archivo labels_eventos.csv creado correctamente.")


✅ Archivo labels_eventos.csv creado correctamente.


In [15]:
# Cargar features y labels
features = pd.read_csv('features_fft_eventos.csv')  # header=None si los datos no tienen nombre de columnas
labels = pd.read_csv('labels_eventos.csv')  # Aquí asumo que labels sí tiene encabezados (tipo_evento, magnitud, vs30, etc.)

# Verificar tamaños
print("Features shape:", features.shape)
print("Labels shape:", labels.shape)

# Importante: verificar que el número de filas coincida
if features.shape[0] != labels.shape[0]:
    raise ValueError("El número de filas entre features y labels no coincide")

# Concatenar features y labels
dataset = pd.concat([features, labels], axis=1)

# Guardar dataset combinado
dataset.to_csv('dataset_completo.csv', index=False)

print("Dataset combinado guardado como 'dataset_completo.csv'")
print(dataset.columns)



Features shape: (1696, 771)
Labels shape: (1696, 9)
Dataset combinado guardado como 'dataset_completo.csv'
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '770', '1 Stiker Slip (SS)', '2 Normal-Oblique', '3 Reverse-Oblique',
       'Mag. 4-6', 'Mag. 6-8', 'Vs30. 0-200', 'Vs30. 200-400', 'Vs30. 400-600',
       'Vs30. 600-'],
      dtype='object', length=780)


In [16]:
dataset
#dataset = pd.read_csv('dataset_completo.csv')

,0,1,2,3,4,5,6,7,8,9,...,770,1 Stiker Slip (SS),2 Normal-Oblique,3 Reverse-Oblique,Mag. 4-6,Mag. 6-8,Vs30. 0-200,Vs30. 200-400,Vs30. 400-600,Vs30. 600-
0,0.073963,0.076551,0.164271,0.189481,0.733871,0.473106,1.069068,2.886660,2.404727,1.237850,...,0.004153,1,0,0,0,1,1,0,0,0
1,0.011226,0.048072,0.099092,0.241767,0.605422,0.387838,1.423007,0.826678,0.427734,0.348254,...,0.001419,1,0,0,0,1,1,0,0,0
2,0.302732,0.580988,0.160099,0.382219,0.508346,1.327788,1.135672,1.770966,2.332856,1.682762,...,0.004402,1,0,0,0,1,1,0,0,0
3,0.204113,0.260953,0.172260,0.289084,0.420757,0.993994,1.422152,1.842656,1.783029,1.889992,...,0.010599,1,0,0,0,1,1,0,0,0
4,0.058527,0.058254,0.166003,0.333401,0.405721,0.439477,0.449853,0.342652,0.654540,0.246920,...,0.001085,1,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1691,0.002611,0.000511,0.000338,0.000140,0.000502,0.001178,0.000035,0.000872,0.000338,0.000767,...,0.000068,0,0,1,1,0,0,0,0,1
1692,0.006713,0.000367,0.000090,0.000441,0.000930,0.003505,0.001380,0.001001,0.000904,0.001561,...,0.000073,0,0,1,1,0,0,0,0,1
1693,0.003659,0.001132,0.001292,0.000670,0.000870,0.001787,0.000959,0.000840,0.000196,0.000615,...,0.000134,0,0,1,1,0,0,0,0,1
1694,0.000671,0.000943,0.000416,0.000725,0.001003,0.003533,0.001582,0.000416,0.001012,0.001010,...,0.000011,0,0,1,1,0,0,0,0,1


In [17]:
# Separar la data


y= dataset[['1 Stiker Slip (SS)', '2 Normal-Oblique', '3 Reverse-Oblique',
    'Mag. 4-6', 'Mag. 6-8',
    'Vs30. 0-200', 'Vs30. 200-400', 'Vs30. 400-600', 'Vs30. 600-']].values
X= dataset.drop(columns=['1 Stiker Slip (SS)', '2 Normal-Oblique', '3 Reverse-Oblique',
    'Mag. 4-6', 'Mag. 6-8',
    'Vs30. 0-200', 'Vs30. 200-400', 'Vs30. 400-600', 'Vs30. 600-']).values

In [18]:
from sklearn.preprocessing import StandardScaler
#Escalar la data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [19]:
import torch
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32) 

In [20]:
import lightning as L
from torch.utils.data import TensorDataset, DataLoader, Subset, random_split


class DataModule(L.LightningDataModule):
    def __init__(self, X, y, batch_size = 64): #De la documentación
        super().__init__()
        # Particiones
        self.X_tensor = X
        self.y_tensor = y
        self.batch_size = batch_size

        # metadata para tu modelo
        self.input_shape = X.shape[1]
        self.num_classes = y.shape[1]

        # placeholders
        self.train_dataset   = None
        self.val_dataset     = None
        self.test_dataset    = None
        self.predict_dataset = None

    def prepare_data(self):
        pass

    def setup(self, stage=None):
        dataset = TensorDataset(self.X_tensor, self.y_tensor)
        n = len(dataset)
        train_size = int(0.8 * n)
        test_size = n - train_size
        seed  = torch.Generator().manual_seed(42)
        self.train_dataset, self.test_dataset = random_split(dataset, [train_size, test_size], generator=seed)
        # solo creamos los datasets con lo que ya tienes
        if stage == "fit" or stage is None:
            trainT_size = int(0.8 * len(self.train_dataset))
            val_size = len(self.train_dataset) - trainT_size
            seed = torch.Generator().manual_seed(42)
            self.train_dataset, self.val_dataset = random_split(self.train_dataset, [trainT_size, val_size], generator=seed)
        
        if stage == "test" or stage is None:
            self.test_dataset = self.test_dataset

        if stage == "predict" or stage is None:
            self.predict_dataset = Subset(self.test_dataset, range(5))

    def train_dataloader(self):
        return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_dataset,batch_size=self.batch_size,shuffle=False)

    def predict_dataloader(self):
        return DataLoader(self.predict_dataset,batch_size=self.batch_size,shuffle=False)



In [21]:
import torch
import torch.nn as nn
import lightning as L
import torchmetrics


class Mymodel2(L.LightningModule):
    def __init__(self, input_shape, num_classes, hidden_units, lambda_L2=0.0, lrate=0.001):
        super().__init__()
        self.lrate = lrate
        self.num_classes = num_classes 
        self.hidden_units = hidden_units  # Store the list of hidden units
        layers = [nn.Flatten()] #flatten layer

        for units in hidden_units: #hidden layers
            layers.append(nn.Linear(input_shape, units))
            layers.append(nn.BatchNorm1d(units))
            layers.append(nn.LeakyReLU(negative_slope=0.01)) #Activcion
            layers.append(nn.Dropout(0.3))
            input_shape = units
        layers.append(nn.Linear(input_shape, num_classes))#output layer, logits 9
        self.classifier = nn.Sequential(*layers) #clasificador

        self.lambda_L2 = lambda_L2  # L2 regularization strength
        self.loss = nn.BCEWithLogitsLoss() #Loss function para problemas multilabel

        self.train_hamming = torchmetrics.HammingDistance(task='multilabel',num_labels=num_classes)
        self.train_f1_macro = torchmetrics.F1Score(task='multilabel',num_labels=num_classes, average='micro')
        self.val_hamming = torchmetrics.HammingDistance(task='multilabel',num_labels=num_classes)
        self.val_f1_macro = torchmetrics.F1Score(task='multilabel',num_labels=num_classes, average='micro')
    
    
    def forward(self, x):
        return self.classifier(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lrate , weight_decay=self.lambda_L2)

    #Que deberia de aumentar en el codigo para poder cumplir con BCEWithLogitsLoss()
    def shared_step(self, batch):
        x, y = batch              # y debe ser un tensor float32 de shape [B, C]
        logits = self(x)          # [B, C]
        loss   = self.loss(logits, y)

        # convertir logits → probabilidades → predicciones binarias
        probs = torch.sigmoid(logits)
        thr   = (self.thresholds.to(probs.device) if hasattr(self,'thresholds')
            else 0.5)
        preds = (probs > thr ).int() #tambien dejale con 0.5 purito a ver que pasa

        return loss, probs,preds, y #probs
    
    def training_step(self, batch, batch_idx):
        loss, probs, preds, labels= self.shared_step(batch)

        ham = self.train_hamming(probs, labels.int()) 
        f1ma=self.train_f1_macro(probs, labels.int())

        self.log('train_loss', loss, logger=True,prog_bar=True)
        self.log('train_ham', ham,  logger=True,prog_bar=True)
        self.log('train_f1mi', f1ma, logger=True,prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, probs, preds, labels= self.shared_step(batch)

        ham = self.val_hamming(probs, labels.int()) 
        f1ma=self.val_f1_macro(probs, labels.int())

        self.log('val_loss', loss, on_step=True, on_epoch=True, logger=True,prog_bar=True)
        self.log('val_ham', ham, on_step=True, on_epoch=True, logger=True,prog_bar=True)
        self.log('val_f1mi', f1ma, on_step=True, on_epoch=True, logger=True,prog_bar=True)
        return loss




In [22]:

from sklearn.model_selection import ParameterGrid

dm = DataModule(X_tensor, y_tensor, batch_size=32)
dm.setup()

param_grid = {
    'hidden_units': [[512,256,128],
    [256,128,64],[256,128,64],
    [256,256,256]],
    'lr': [0.001, 0.0005, 0.01, 0.011],
    'lambda_L2': [0, 1e-5, 1e-4, 1e-3]
}

grid = ParameterGrid(param_grid)
results = []

for p in grid:
    model = Mymodel2(
        input_shape=dm.input_shape,
        num_classes=dm.num_classes,
        hidden_units=p['hidden_units'],
        lambda_L2=p['lambda_L2'],
        lrate=p['lr']  
    )

    trainer = L.Trainer(max_epochs=30, accelerator="gpu" if torch.cuda.is_available() else "cpu", devices=1)
    trainer.fit(model,dm)
    val_metrics = trainer.validate(model, dataloaders=dm.val_dataloader())
    val_loss = val_metrics[0]['val_ham_epoch'] 
    results.append((p, val_loss))
 
best_params = sorted(results, key=lambda x: x[1])[0]
print(f"\nMejores parámetros: {best_params}")

/home/mare/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name           | Type                      | Params | Mode 
---------------------------------------------------------------------
0 | classifier     | Sequential                | 562 K  | train
1 | loss           | BCEWithLogitsLoss         | 0      | train
2 | train_hamming  | MultilabelHammingDistance | 0      | train
3 | train_f1_macro | MultilabelF1Score         | 0      | train
4 | val_hamming    | MultilabelHammingDistance | 0      | train
5 | val_f1_macro   | MultilabelF1Score         | 0      | train
---------------------------------------------------------------------
562 K     Traina

/home/mare/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/mare/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/mare/.local/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (34) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 29: 100%|██████████| 34/34 [00:01<00:00, 29.25it/s, v_num=3, train_loss=0.433, train_ham=0.190, train_f1mi=0.652, val_loss_step=0.566, val_ham_step=0.271, val_f1mi_step=0.435, val_loss_epoch=0.631, val_ham_epoch=0.317, val_f1mi_epoch=0.372]

`Trainer.fit` stopped: `max_epochs=30` reached.


Validation DataLoader 0: 100%|██████████| 9/9 [00:00<00:00, 93.88it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     val_f1mi_epoch         0.37198641896247864
      val_ham_epoch         0.3174019753932953
     val_loss_epoch         0.6309099793434143
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


...

Validation DataLoader 0: 100%|██████████| 9/9 [00:00<00:00, 66.46it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     val_f1mi_epoch         0.3044079542160034
      val_ham_epoch         0.3133169710636139
     val_loss_epoch         0.5967991948127747
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name           | Type                      | Params | Mode 
---------------------------------------------------------------------
0 | classifier     | Sequential                | 333 K  | train
1 | loss           | BCEWithLogitsLoss         | 0      | train
2 | train_hamming  | MultilabelHammingDistance | 0      | train
3 | train_f1_macro | MultilabelF1Score         | 0      | train
4 | val_hamming    | MultilabelHammingDistance | 0      | train
5 | val_f1_macro   | MultilabelF1Score         | 0      | train
---------------------------------------------------------------------
333 K     Trainable params
0         Non-trainable params
333 K     Total params
1.332     Total estimated model params size (MB)
20        Modules in train mode
0    

Epoch 29: 100%|██████████| 34/34 [00:00<00:00, 35.31it/s, v_num=66, train_loss=0.589, train_ham=0.310, train_f1mi=0.316, val_loss_step=0.570, val_ham_step=0.306, val_f1mi_step=0.312, val_loss_epoch=0.597, val_ham_epoch=0.313, val_f1mi_epoch=0.297]

`Trainer.fit` stopped: `max_epochs=30` reached.


Validation DataLoader 0: 100%|██████████| 9/9 [00:00<00:00, 57.78it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     val_f1mi_epoch         0.2972668707370758
      val_ham_epoch         0.31290850043296814
     val_loss_epoch         0.5974172353744507
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Mejores parámetros: ({'hidden_units': [256, 128, 64], 'lambda_L2': 1e-05, 'lr': 0.011}, 0.30106207728385925)


In [24]:
best_params_dict, best_params = sorted(results, key=lambda x: x[1])[0]
print(f"\nMejores parámetros: {best_params}")


Mejores parámetros: 0.30106207728385925


In [28]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import RepeatedKFold
import pandas as pd
from scipy.stats import ttest_1samp
import lightning as L  

# Parámetros globales
SEED        = 42
BATCH_SIZE  = 32
N_SPLITS    = 5
N_REPEATS   = 2
MAX_EPOCHS  = 100

# Valores óptimos (obtenidos previamente)
best_cfg = {
    'hidden_units': best_params_dict['hidden_units'],
    'lambda_L2':    best_params_dict['lambda_L2']
}

# Helper para crear DataLoader a partir de arrays
def make_loader(X_arr, y_arr, batch_size, shuffle=False):
    X_t = torch.tensor(X_arr, dtype=torch.float32)
    y_t = torch.tensor(y_arr, dtype=torch.float32)
    ds  = TensorDataset(X_t, y_t)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)

# Configuramos RepeatedKFold
cv = RepeatedKFold(n_splits=N_SPLITS, n_repeats=N_REPEATS, random_state=SEED)

# Acumulador de resultados
records = []

for fold_idx, (tr_idx, vl_idx) in enumerate(cv.split(X), start=1):
    print(f"▶ Fold {fold_idx}/{N_SPLITS * N_REPEATS}")
    
    # 1) DataLoaders por fold
    train_loader = make_loader(X[tr_idx], y[tr_idx], BATCH_SIZE, shuffle=True)
    val_loader   = make_loader(X[vl_idx], y[vl_idx], BATCH_SIZE)
    
    # 2) Crear un modelo nuevo para este fold
    model = Mymodel2(
        input_shape=X.shape[1],
        num_classes=y.shape[1],
        hidden_units=best_cfg['hidden_units'],
        lambda_L2=best_cfg['lambda_L2']
    )
    
    # 3) Callback de checkpoint
    ckpt = L.pytorch.callbacks.ModelCheckpoint(
        dirpath='models',
        filename=f'fold{fold_idx}_best',
        monitor='val_f1mi_epoch',
        mode='max',
        save_top_k=1
    )
    
    # 4) Trainer
    trainer = L.Trainer(
        max_epochs=MAX_EPOCHS,
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=1,
        callbacks=[ckpt],
        logger=True,
        enable_progress_bar=True
    )
    
    trainer.fit(model, train_loader, val_loader)
    metrics = trainer.validate(model, val_loader, verbose=False)[0]
    metrics['fold'] = fold_idx
    records.append(metrics)

# 6) Results → DataFrame
df_results = pd.DataFrame(records)
print("\nResultados por fold:\n", df_results)

# 7) Resumen estadístico: mean, std y p-value (t-test vs 0)
summary = pd.DataFrame({
    'mean':   df_results.mean(),
    'std':    df_results.std(),
    'p-value': [ttest_1samp(df_results[col], popmean=0).pvalue 
                for col in df_results.columns]
})
print("\nResumen:\n", summary)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name           | Type                      | Params | Mode 
---------------------------------------------------------------------
0 | classifier     | Sequential                | 240 K  | train
1 | loss           | BCEWithLogitsLoss         | 0      | train
2 | train_hamming  | MultilabelHammingDistance | 0      | train
3 | train_f1_macro | MultilabelF1Score         | 0      | train
4 | val_hamming    | MultilabelHammingDistance | 0      | train
5 | val_f1_macro   | MultilabelF1Score         | 0      | train
---------------------------------------------------------------------
240 K     Trainable params
0         Non-trainable params
240 K     Total params
0.961     Total estimated model params size (MB)
20        Modules in train mode
0         Modules in eval mode


▶ Fold 1/10
Epoch 99: 100%|██████████| 43/43 [00:00<00:00, 55.88it/s, v_num=68, train_loss=0.330, train_ham=0.176, train_f1mi=0.698, val_loss_step=0.575, val_ham_step=0.267, val_f1mi_step=0.478, val_loss_epoch=0.618, val_ham_epoch=0.303, val_f1mi_epoch=0.417]

`Trainer.fit` stopped: `max_epochs=100` reached.


Validation DataLoader 0: 100%|██████████| 11/11 [00:00<00:00, 176.00it/s]
▶ Fold 2/10


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/mare/.local/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/mare/Documents/Documentos/6. Sexto Semestre/Machine Learning/Proyecto3/models exists and is not empty.

  | Name           | Type                      | Params | Mode 
---------------------------------------------------------------------
0 | classifier     | Sequential                | 240 K  | train
1 | loss           | BCEWithLogitsLoss         | 0      | train
2 | train_hamming  | MultilabelHammingDistance | 0      | train
3 | train_f1_macro | MultilabelF1Score         | 0      | train
4 | val_hamming    | MultilabelHammingDistance | 0      | train
5 | val_f1_macro   | MultilabelF1Score         | 0      | train
---------------------------------------------------------------------
240 K     Trainable params
0         Non-trainable params
240 K     To

...

Validation DataLoader 0: 100%|██████████| 11/11 [00:00<00:00, 86.24it/s]
▶ Fold 10/10


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name           | Type                      | Params | Mode 
---------------------------------------------------------------------
0 | classifier     | Sequential                | 240 K  | train
1 | loss           | BCEWithLogitsLoss         | 0      | train
2 | train_hamming  | MultilabelHammingDistance | 0      | train
3 | train_f1_macro | MultilabelF1Score         | 0      | train
4 | val_hamming    | MultilabelHammingDistance | 0      | train
5 | val_f1_macro   | MultilabelF1Score         | 0      | train
---------------------------------------------------------------------
240 K     Trainable params
0         Non-trainable params
240 K     Total params
0.961     Total estimated model params size (MB)
20        Modules in train mode
0         Modules in eval mode


Epoch 99: 100%|██████████| 43/43 [00:01<00:00, 27.10it/s, v_num=77, train_loss=0.367, train_ham=0.171, train_f1mi=0.697, val_loss_step=0.552, val_ham_step=0.257, val_f1mi_step=0.511, val_loss_epoch=0.634, val_ham_epoch=0.291, val_f1mi_epoch=0.458]

`Trainer.fit` stopped: `max_epochs=100` reached.


Validation DataLoader 0: 100%|██████████| 11/11 [00:00<00:00, 61.79it/s]

Resultados por fold:
    val_loss_epoch  val_ham_epoch  val_f1mi_epoch  fold
0        0.618286       0.302614        0.417194     1
1        0.639576       0.289086        0.463071     2
2        0.652160       0.308423        0.421859     3
3        0.627313       0.282858        0.467804     4
4        0.656392       0.294985        0.447720     5
5        0.632891       0.302941        0.428448     6
6        0.646732       0.301540        0.431844     7
7        0.672276       0.302196        0.445616     8
8        0.633980       0.298591        0.445667     9
9        0.634084       0.291380        0.457610    10

Resumen:
                     mean       std       p-value
val_loss_epoch  0.641369  0.015774  5.288990e-16
val_ham_epoch   0.297461  0.007772  9.111591e-16
val_f1mi_epoch  0.442683  0.017371  3.532430e-14
fold            5.500000  3.027650  2.781960e-04


In [29]:
# Esto para que es:
# escoger el mejor modelo
val_metrics = trainer.validate(model, dataloaders=dm.val_dataloader(), verbose=False)
print(f"Validation Loss: {val_metrics}")
test_metrics = trainer.validate(model, dataloaders=dm.test_dataloader(), verbose=False)
print(f"Test Loss: {test_metrics}")
train_metrics = trainer.validate(model, dataloaders=dm.train_dataloader(), verbose=False)
print(f"Train Loss: {train_metrics}")


/home/mare/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Validation DataLoader 0: 100%|██████████| 9/9 [00:00<00:00, 130.03it/s]
Validation Loss: [{'val_loss_epoch': 0.8049192428588867, 'val_ham_epoch': 0.31209152936935425, 'val_f1mi_epoch': 0.4522566795349121}]
Validation DataLoader 0: 100%|██████████| 11/11 [00:00<00:00, 60.82it/s]
Test Loss: [{'val_loss_epoch': 0.8704195022583008, 'val_ham_epoch': 0.3009803891181946, 'val_f1mi_epoch': 0.4775489866733551}]


/home/mare/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:476: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Validation DataLoader 0: 100%|██████████| 43/43 [00:00<00:00, 78.53it/s] 
Train Loss: [{'val_loss_epoch': 0.8665755391120911, 'val_ham_epoch': 0.3112913966178894, 'val_f1mi_epoch': 0.45870381593704224}]


In [31]:
%reload_ext tensorboard
%tensorboard --logdir "lightning_logs"


Reusing TensorBoard on port 6006 (pid 23950), started 0:00:14 ago. (Use '!kill 23950' to kill it.)